In [3]:
import os
from PIL import Image

# 设定webp图片所在目录
webp_dir = '/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps'

# 获取所有webp文件名，并按文件名排序
webp_files = [f for f in os.listdir(webp_dir) if f.endswith('.webp')]
webp_files.sort()  # 按文件名升序排列

# 读取所有webp图片
images = [Image.open(os.path.join(webp_dir, f)) for f in webp_files]

# 中文注释：每4张图片拼接为一张竖直拼接的大图
group_size = 4  # 每组拼接4张
merged_images = []  # 存储每组拼接后的图片

for i in range(0, len(images), group_size):
    group = images[i:i+group_size]
    # 获取本组图片的最大宽度和总高度
    widths, heights = zip(*(img.size for img in group))
    total_width = max(widths)
    total_height = sum(heights)
    # 统一图片模式，避免webp编码错误
    if group:
        mode = group[0].mode
        if mode not in ['RGB', 'RGBA']:
            mode = 'RGB'
            group = [img.convert('RGB') for img in group]
    else:
        mode = 'RGB'
    # 创建空白图片用于拼接
    merged_img = Image.new(mode, (total_width, total_height))  # 中文注释：创建合并后的空白图片
    y_offset = 0
    for img in group:
        # 中文注释：如果单张图片模式与合并图片不一致，进行转换
        if img.mode != mode:
            img = img.convert(mode)
        merged_img.paste(img, (0, y_offset))
        y_offset += img.size[1]
    merged_images.append(merged_img)

# 保存每组拼接后的图片为webp格式，文件名为all_0.webp, all_1.webp, ...
for idx, merged_img in enumerate(merged_images):
    output_path = os.path.join(webp_dir, f'all_{idx}.webp')
    try:
        merged_img.save(output_path, 'WEBP')  # 中文注释：保存为webp格式
        print(f"已将第{idx+1}组的{group_size}张webp图片竖直拼接为: {output_path}")  # 中文注释：输出合并结果
    except Exception as e:
        print(f"保存webp图片时出错: {e}")  # 中文注释：捕获并输出保存错误


已将第1组的4张webp图片竖直拼接为: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps/all_0.webp
已将第2组的4张webp图片竖直拼接为: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps/all_1.webp


In [ ]:
import cv2  # 中文注释：导入OpenCV库用于视频写入
import numpy as np
importos
webp_dir = '/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps'
# 中文注释：将webp目录下前4个webp图片拼接为一张，后4个拼接为一张，拼接后分别生成动态webp（动图）和4K分辨率的mp4视频
webp_files = [f for f in os.listdir(webp_dir) if f.endswith('.webp')]
webp_files.sort()  # 中文注释：按文件名升序排列

# 中文注释：分组，每组4张
group_size = 4
groups = [webp_files[:4], webp_files[4:8]]

output_names = ['all_0.webp', 'all_1.webp']
video_names = ['all_0.mp4', 'all_1.mp4']
animated_names = ['all_0_animated.webp', 'all_1_animated.webp']

for idx, group in enumerate(groups):
    # 中文注释：读取本组图片
    imgs = [Image.open(os.path.join(webp_dir, f)) for f in group]
    # 中文注释：统一图片模式
    mode = imgs[0].mode
    if mode not in ['RGB', 'RGBA']:
        imgs = [img.convert('RGB') for img in imgs]
        mode = 'RGB'
    # 中文注释：竖直拼接
    widths, heights = zip(*(img.size for img in imgs))
    total_width = max(widths)
    total_height = sum(heights)
    merged_img = Image.new(mode, (total_width, total_height))
    y_offset = 0
    for img in imgs:
        if img.mode != mode:
            img = img.convert(mode)
        merged_img.paste(img, (0, y_offset))
        y_offset += img.size[1]
    # 中文注释：保存拼接后的webp图片
    merged_path = os.path.join(webp_dir, output_names[idx])
    merged_img.save(merged_path, 'WEBP')
    print(f"已将第{idx+1}组的4张webp图片竖直拼接为: {merged_path}")

    # ========== 新增：生成动态webp（动图） ==========
    # 中文注释：将本组4张图片竖直拼接后，依次作为帧生成动态webp
    frames = []
    for i in range(len(imgs)):
        # 竖直拼接前i+1张图片，剩下的用白色填充
        temp_img = Image.new(mode, (total_width, total_height), color=(255,255,255) if mode=='RGB' else (255,255,255,0))
        y_offset = 0
        for j in range(i+1):
            img = imgs[j]
            if img.mode != mode:
                img = img.convert(mode)
            temp_img.paste(img, (0, y_offset))
            y_offset += img.size[1]
        frames.append(temp_img)
    # 中文注释：最后一帧为完整拼接
    frames.append(merged_img)
    # 中文注释：保存为动态webp，帧率10fps，循环播放
    animated_path = os.path.join(webp_dir, animated_names[idx])
    frames[0].save(animated_path, save_all=True, append_images=frames[1:], duration=200, loop=0, format='WEBP')
    print(f"已生成动态webp动图: {animated_path}")

    # ========== 生成4K分辨率mp4视频 ==========
    # 中文注释：将每一帧转换为OpenCV格式，缩放到4K分辨率
    max_width, max_height = 3840, 2160  # 4K分辨率
    cv_frames = []
    for frame in frames:
        # PIL转OpenCV格式
        frame_cv = np.array(frame)
        if frame_cv.shape[-1] == 4:
            frame_cv = cv2.cvtColor(frame_cv, cv2.COLOR_RGBA2BGR)
        else:
            frame_cv = cv2.cvtColor(frame_cv, cv2.COLOR_RGB2BGR)
        h, w = frame_cv.shape[:2]
        scale = min(max_width / w, max_height / h, 1.0)
        new_w, new_h = int(w * scale), int(h * scale)
        if scale < 1.0:
            frame_cv = cv2.resize(frame_cv, (new_w, new_h), interpolation=cv2.INTER_AREA)
        cv_frames.append(frame_cv)
    # 中文注释：写入mp4视频，帧率10fps，循环一次
    video_path = os.path.join(webp_dir, video_names[idx])
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 2
    out = cv2.VideoWriter(video_path, fourcc, fps, (cv_frames[0].shape[1], cv_frames[0].shape[0]))
    for frame in cv_frames:
        out.write(frame)
    out.release()
    print(f"已将图片 {merged_path} 合成为动态4K mp4视频: {video_path}")


NameError: name 'os' is not defined

In [3]:
import os
import subprocess

# WebP 动图目录
webp_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps"

# 输出视频目录
output_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/videos"
os.makedirs(output_dir, exist_ok=True)

# 获取所有 webp 文件并按名字自然排序
webp_files = [f for f in os.listdir(webp_dir) if f.endswith(".webp")]
# 每 4 个文件生成一个视频
chunk_size = 4
for i in range(0, len(webp_files), chunk_size):
    chunk = webp_files[i:i+chunk_size]
    # 创建一个临时 txt 文件，ffmpeg 读取顺序
    list_file = os.path.join(webp_dir, f"list_{i//chunk_size}.txt")
    with open(list_file, "w") as f:
        for webp in chunk:
            f.write(f"file '{os.path.join(webp_dir, webp)}'\n")

    output_video = os.path.join(output_dir, f"video_{i//chunk_size + 1}.mp4")
    
    # 调用 ffmpeg 合并 webp 成 mp4
    cmd = [
        "ffmpeg",
        "-y",  # 覆盖同名文件
        "-f", "concat",
        "-safe", "0",
        "-i", list_file,
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        output_video
    ]
    subprocess.run(cmd)
    print(f"生成视频: {output_video}")


生成视频: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/videos/video_1.mp4
生成视频: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/videos/video_2.mp4


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

SyntaxError: invalid syntax (1792314964.py, line 4)

In [5]:
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips

webp_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps"
output_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/videos"
os.makedirs(output_dir, exist_ok=True)

# 获取 webp 文件并按名字排序
webp_files = [f for f in os.listdir(webp_dir) if f.endswith(".webp")]

# 每 4 个生成一个视频
chunk_size = 4
for i in range(0, len(webp_files), chunk_size):
    chunk = webp_files[i:i+chunk_size]
    clips = []
    
    for webp_file in chunk:
        path = os.path.join(webp_dir, webp_file)
        clip = VideoFileClip(path)  # moviepy 自动处理 WebP 动画
        clips.append(clip)
    
    final_clip = concatenate_videoclips(clips, method="compose")
    output_path = os.path.join(output_dir, f"video_{i//chunk_size + 1}.mp4")
    final_clip.write_videofile(output_path, codec="libx264", fps=25)
    final_clip.close()
    
    # 关闭每个子 clip
    for c in clips:
        c.close()

    print(f"生成视频: {output_path}")


ModuleNotFoundError: No module named 'moviepy'

In [7]:
import os
import subprocess

webp_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps"
output_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/videos"
os.makedirs(output_dir, exist_ok=True)

# 获取所有 webp 文件并自然排序
webp_files = [f for f in os.listdir(webp_dir) if f.endswith(".webp")]

chunk_size = 4

for i in range(0, len(webp_files), chunk_size):
    chunk = webp_files[i:i+chunk_size]

    # 拼接输入参数，直接让 ffmpeg 串联 WebP 动画
    # 每个 WebP 动画用 -i 单独输入，后面再 concat filter
    input_args = []
    for f in chunk:
        input_args += ["-i", os.path.join(webp_dir, f)]

    # 构建 filter_complex concat
    filter_inputs = "".join([f"[{j}:v:0]" for j in range(len(chunk))])
    filter_complex = f"{filter_inputs}concat=n={len(chunk)}:v=1:a=0[outv]"

    output_path = os.path.join(output_dir, f"video_{i//chunk_size + 1}.mp4")

    cmd = ["ffmpeg", "-y"] + input_args + [
        "-filter_complex", filter_complex,
        "-map", "[outv]",
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        output_path
    ]

    subprocess.run(cmd)
    print(f"生成视频: {output_path}")


生成视频: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/videos/video_1.mp4
生成视频: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/videos/video_2.mp4


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [16]:
import os
import subprocess

gif_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps"
output_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps"
os.makedirs(output_dir, exist_ok=True)

# 中文注释：将每个 gif 单独转为一个 mp4 视频
gif_files = [f for f in os.listdir(gif_dir) if f.endswith(".gif")]

for idx, gif_file in enumerate(gif_files):
    input_path = os.path.join(gif_dir, gif_file)
    output_path = os.path.join(output_dir, f"video_{idx + 3}.mp4")
    
    # 中文注释：使用 ffmpeg 将 gif 转为 mp4，设置帧率为24
    cmd = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-vf", "setpts=3*PTS",  # 时长拉长 3 倍
        "-r", "24",             # 目标帧率 24
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        output_path
    ]
    subprocess.run(cmd)
    print(f"生成视频: {output_path}")


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

生成视频: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps/video_3.mp4


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

生成视频: /home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps/video_4.mp4


In [ ]:
import os
import subprocess
from multiprocessing import Pool

gif_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps"
output_dir = "/home/hongyuan/obj_motion/Trans4Motion.github.io/static/webps"
os.makedirs(output_dir, exist_ok=True)

# 中文注释：将每个 gif 单独转为一个 mp4 视频，使用多进程加速
gif_files = [f for f in os.listdir(gif_dir) if f.endswith(".gif")]

def convert_gif_to_mp4(args):
    idx, gif_file = args
    input_path = os.path.join(gif_dir, gif_file)
    output_path = os.path.join(output_dir, f"video_{idx + 3}.mp4")
    # 中文注释：使用 ffmpeg 将 gif 转为 mp4，设置帧率为24，时长拉长3倍
    cmd = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-vf", "setpts=3*PTS",  # 时长拉长 3 倍
        "-r", "24",             # 目标帧率 24
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        output_path
    ]
    subprocess.run(cmd)
    print(f"生成视频: {output_path}")

if __name__ == "__main__":
    # 中文注释：使用进程池并发处理所有 gif 文件
    with Pool() as pool:
        pool.map(convert_gif_to_mp4, list(enumerate(gif_files)))
